scikit-learn (sklearn)은 다양한 머신러닝 알고리즘 및 데이터 처리 모듈들을 가지고 있는 툴킷입니다. sklearn의 CountVectorizer를 이용하여 term frequency matrix를 만드는 법을 연습합니다. 

먼저, 앞선 연습에서 만들어둔 corpus를 읽어와 koNLPy의 Twitter 형태소 분석기를 이용하여 명사를 추출하는 custom tokenizer를 만들겠습니다. 

In [1]:
preprocessed_data_fname = './tmp/reviews.txt'
model_folder = './tmp/'

In [3]:
idxs = []
scores = []
texts = []
with open(preprocessed_data_fname, encoding='utf-8') as f:
    for doc in f:
        try:
            idx, score, text = doc.split('\t')
            score = int(score)
            idxs.append(idx)
            scores.append(score)
            texts.append(text)
        except:
            continue
        
len(idxs), len(scores), len(texts)

(27689, 27689, 27689)

Python에서는 두 개의 for loop을 한 줄에 표현하는 list comprehension 표현이 있습니다. 이에 익숙해지면 Python 코드를 명료하고 간결하게 작성할 수 있습니다. 

list l1은 [ ['a', 'b'], [1, 2, 3] ] 처럼 두 개의 list를 가지고 있습니다. 이 두 개의 리스트의 모든 값을 일렬로 나열한 하나의 flatten list를 만들어봅시다. 

In [4]:
l1 = [ ['a', 'b'], [1, 2, 3] ]
print(l1)

[['a', 'b'], [1, 2, 3]]


아래의 구문은 이렇게 해석할 수 있습니다. 

1. element 들을 하나루 묶은 list를 생성할 겁니다. [element]
1. 그런데 이 element는 [1, 2, 3]과 같은 small_list 안의 원소를 의미합니다. 이는 다음과 같이 정의합니다. 

    for element in small_list
    
1. 그러나 small_list는 아직 정의된 적이 없습니다. element가 정의되기 전에 small_list가 정의되어야 합니다. 그렇기 때문에 for element in small_list 앞에서 small_list를 먼저 정의합니다. 

    for small_list in l1 for element in small_list
   

In [5]:
flatten_l1 = [element for small_list in l1 for element in small_list]
print(flatten_l1)

['a', 'b', 1, 2, 3]


위 코드는 아래와 동일합니다. 네 줄로 적어야 하는 부분이 한 줄로 표현됩니다. 

In [6]:
flatten_l1 = []
for small_list in l1:
    for element in small_list:
        flatten_l1.append(element)
print(flatten_l1)

['a', 'b', 1, 2, 3]


## From corpus to noun term frequency sparse matrix

Twitter.nouns(sent) 는 주어진 sent에서 명사를 추출하여 return 합니다. type은 list of str입니다. 

In [7]:
from collections import defaultdict
from pprint import pprint
from konlpy.tag import Twitter

twitter = Twitter()
twitter.nouns('이건 테스트입니다')

['이건', '테스트']

a line a document이기 때문에 docs[i]은 하나의 문서를 의미하며, 하나의 문서는 두 칸 띄어쓰기로 줄이 나뉘어져 있습니다. 그러나 우리는 지금 모든 문서에서 명사를 추출한 뒤, 각 명사들이 몇 번 나왔었는지를 알아보려 합니다. 그러므로 한 줄 한 줄을 굳이 나누지는 않겠습니다. 

앞서 배운 collections.Counter를 이용하여 한 번에 명사의 개수를 세어 봅시다. 

In [8]:
from collections import Counter

noun_counter = Counter([noun for text in texts for noun in twitter.nouns(text)])
print('num of nouns: %d' % len(noun_counter))

num of nouns: 11758


term frequency matrix를 만들 때, 거의 등장하지 않는 단어들은 의미가 없습니다. min count를 설정하여 각 threshold 별로 몇 개의 명사가 살아남는지 알아봅니다. 

In [9]:
for min_count in [2, 3, 5, 10]:
    _counter = {word for word, freq in noun_counter.items() if freq >= min_count}
    print('num of nouns (min_count = %d): %d' % (min_count, len(_counter)))

num of nouns (min_count = 2): 5847
num of nouns (min_count = 3): 4182
num of nouns (min_count = 5): 2879
num of nouns (min_count = 10): 1715


우리는 5번 이상 나오는 4102개의 명사를 이용하여 term frequency matrix를 만들어보겠습니다. CountVectorizer는 문서가 주어지면 띄어쓰기를 tokenizer로 이용합니다. filtering이 되지 않습니다. 이를 방지하기 위하여 빈도수가 5 이상인 명사만을 return 하는 custom tokenizer를 만들어 봅시다. 

Twitter.nouns()와 custom_tokenizer의 결과가 달라짐을 확인할 수 있습니다.

In [13]:
noun_dict = {word for word, freq in noun_counter.items() if freq >= 5}

def custom_tokenizer(doc):
    return [word for word in twitter.nouns(doc) if word in noun_dict]

print(twitter.nouns(texts[2]))
print(custom_tokenizer(texts[2]))

['제목', '덕필']
['제목']


In [14]:
len(noun_dict)

2879

CountVectorizer는 document frequency를 기준으로 대부분 문서에 등장하거나 거의 등장하지 않는 극단적인 단어들을 제거할 수 있습니다. min_df와 max_df를 이용하면 되며, df는 비율입니다. [0, 1] 사이의 값을 입력해야 합니다. 

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=custom_tokenizer, min_df=0.001, max_df=0.95)
x_sparse = vectorizer.fit_transform(texts)

x_sparse는 scipy의 sparse matrix입니다. Sparse matrix는 0 값이 많은 행렬에 대하여 0이 아닌 (i, j)의 값만을 저장한 matrix를 의미합니다. Term frequency matrix는 very sparse matrix에 속하므로 dense matrix보다 sparse matrix가 훨씬 효율적입니다. dense matrix는 문서의 양이 조금만 커도 메모리가 폭발합니다. 

In [21]:
x_sparse

<27689x785 sparse matrix of type '<class 'numpy.int64'>'
	with 104396 stored elements in Compressed Sparse Row format>

## CountVectorizer에서 각 column에 해당하는 term 알아내기

vectorizer.vocabulary_ 는 dict 형식으로, {word:index} 입니다. 

이를 이용하여 vocab to index, index to vocab을 만들어 봅니다. 정부라는 단어가 1663에 해당합니다. 

In [22]:
vocab2int = vectorizer.vocabulary_
vocab2int['영화']

480

index는 0부터 시작합니다. 그러므로 int2vocab은 dictionary 형태보다 list 형태로 가지고 있어도 좋습니다. 이를 위해 sorted 함수를 이용합니다. 1663 번째 단어가 정부임을 다시 확인할 수 있습니다. 

In [23]:
int2vocab = [word for word,index in sorted(vocab2int.items(), key=lambda x:x[1])]
int2vocab[97]

'끝'

## Sparse matrix I/O

Sparse matrix를 파일로 저장하고 부르는 것은 아래와 같습니다.

In [24]:
from scipy.io import mmwrite, mmread

mmwrite('./tmp/x.mm', x_sparse)
loaded_x_sparse = mmread('./tmp/x.mm')
loaded_x_sparse

<27689x785 sparse matrix of type '<class 'numpy.int64'>'
	with 104396 stored elements in COOrdinate format>